<a href="https://colab.research.google.com/github/Dhruv-Limbani/Financial-Sentiment-Analysis-using-NLP/blob/master/BERT_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_csv('/content/Preprocessed_Data.csv')
df

,Sentence,Sentiment,text_length,cleaned_text,tokenized_text,word_lengths,document_length,bigrams,trigrams,avg_word_length
0,The GeoSolutions technology will leverage Bene...,positive,218,geosolutions technology leverage benefon gps s...,"['geosolutions', 'technology', 'leverage', 'be...","[12, 10, 8, 7, 3, 8, 9, 8, 5, 6, 10, 9, 8, 8, ...",21,"[('geosolutions', 'technology'), ('technology'...","[('geosolutions', 'technology', 'leverage'), (...",7.714286
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,55,esi low 1 50 2 50 bk real possibility,"['esi', 'low', '1', '50', '2', '50', 'bk', 're...","[3, 3, 1, 2, 1, 2, 2, 4, 11]",9,"[('esi', 'low'), ('low', '1'), ('1', '50'), ('...","[('esi', 'low', '1'), ('low', '1', '50'), ('1'...",3.222222
2,"For the last quarter of 2010 , Componenta 's n...",positive,193,last quarter 2010 componenta net sale doubled ...,"['last', 'quarter', '2010', 'componenta', 'net...","[4, 7, 4, 10, 3, 4, 7, 7, 6, 6, 4, 7, 5, 4, 3,...",21,"[('last', 'quarter'), ('quarter', '2010'), ('2...","[('last', 'quarter', '2010'), ('quarter', '201...",5.000000
3,According to the Finnish-Russian Chamber of Co...,neutral,128,according finnish russian chamber commerce maj...,"['according', 'finnish', 'russian', 'chamber',...","[9, 7, 7, 7, 8, 5, 12, 7, 7, 9, 6]",11,"[('according', 'finnish'), ('finnish', 'russia...","[('according', 'finnish', 'russian'), ('finnis...",7.636364
4,The Swedish buyout firm has sold its remaining...,neutral,135,swedish buyout firm sold remaining 22 4 percen...,"['swedish', 'buyout', 'firm', 'sold', 'remaini...","[7, 6, 4, 4, 9, 2, 1, 7, 5, 6, 8, 5, 6, 7, 6, 7]",16,"[('swedish', 'buyout'), ('buyout', 'firm'), ('...","[('swedish', 'buyout', 'firm'), ('buyout', 'fi...",5.625000
...,...,...,...,...,...,...,...,...,...,...
5837,RISING costs have forced packaging producer Hu...,negative,107,rising cost forced packaging producer huhtamak...,"['rising', 'cost', 'forced', 'packaging', 'pro...","[6, 4, 6, 9, 8, 9, 3, 2, 3, 9, 13, 5]",12,"[('rising', 'cost'), ('cost', 'forced'), ('for...","[('rising', 'cost', 'forced'), ('cost', 'force...",6.416667
5838,Nordic Walking was first used as a summer trai...,neutral,83,nordic walking first used summer training meth...,"['nordic', 'walking', 'first', 'used', 'summer...","[6, 7, 5, 4, 6, 8, 6, 5, 7, 5]",10,"[('nordic', 'walking'), ('walking', 'first'), ...","[('nordic', 'walking', 'first'), ('walking', '...",5.900000
5839,"According shipping company Viking Line , the E...",neutral,99,according shipping company viking line eu deci...,"['according', 'shipping', 'company', 'viking',...","[9, 8, 7, 6, 4, 2, 8, 11, 9, 6]",10,"[('according', 'shipping'), ('shipping', 'comp...","[('according', 'shipping', 'company'), ('shipp...",7.000000
5840,"In the building and home improvement trade , s...",neutral,88,building home improvement trade sale decreased...,"['building', 'home', 'improvement', 'trade', '...","[8, 4, 11, 5, 4, 9, 2, 1, 3, 3, 1, 2]",12,"[('building', 'home'), ('home', 'improvement')...","[('building', 'home', 'improvement'), ('home',...",4.416667


In [3]:
df.dropna(inplace=True)

In [4]:
label_encoder = LabelEncoder()
df['Sentiment_encoded'] = label_encoder.fit_transform(df['Sentiment'])
y = df['Sentiment_encoded']
y

,Sentiment_encoded
0,2
1,0
2,2
3,1
4,1
...,...
5837,0
5838,1
5839,1
5840,1


In [5]:
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

Label Mapping: {'negative': 0, 'neutral': 1, 'positive': 2}


In [6]:
x_train_text, x_test_text, y_train, y_test = train_test_split(df['cleaned_text'], y, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_and_pad(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')

x_train_tokens = tokenize_and_pad(x_train_text)
x_test_tokens = tokenize_and_pad(x_test_text)

x_train_tokens,x_test_tokens

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


({'input_ids': <tf.Tensor: shape=(4672, 111), dtype=int32, numpy=
 array([[  101,  2632,  2850, ...,     0,     0,     0],
        [  101,  6599,  3473, ...,     0,     0,     0],
        [  101,  2095, 11443, ...,     0,     0,     0],
        ...,
        [  101,  6207,  2471, ...,     0,     0,     0],
        [  101,  9089,  2063, ...,     0,     0,     0],
        [  101,  8915,  9363, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4672, 111), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4672, 111), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..

In [7]:
x_train_tokens['input_ids']

<tf.Tensor: shape=(4672, 111), dtype=int32, numpy=
array([[  101,  2632,  2850, ...,     0,     0,     0],
       [  101,  6599,  3473, ...,     0,     0,     0],
       [  101,  2095, 11443, ...,     0,     0,     0],
       ...,
       [  101,  6207,  2471, ...,     0,     0,     0],
       [  101,  9089,  2063, ...,     0,     0,     0],
       [  101,  8915,  9363, ...,     0,     0,     0]], dtype=int32)>

In [8]:
x_train_tokens['attention_mask']

<tf.Tensor: shape=(4672, 111), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [9]:
def create_model():
    input_ids = tf.keras.Input(shape=(111,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.Input(shape=(111,), dtype=tf.int32, name='attention_mask')

    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    outputs = bert_model(input_ids, attention_mask=attention_mask)[1]

    x = tf.keras.layers.Dense(111, activation='relu')(outputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
model = create_model()
history = model.fit(
    {'input_ids': x_train_tokens['input_ids'], 'attention_mask': x_train_tokens['attention_mask']},
    y_train,
    epochs=3,
    batch_size=16,
    validation_split=0.1
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
263/263 [==============================] - 145s 391ms/step - loss: 0.8978 - accuracy: 0.5718 - val_loss: 0.6410 - val_accuracy: 0.7137
Epoch 2/3
263/263 [==============================] - 101s 386ms/step - loss: 0.6230 - accuracy: 0.7329 - val_loss: 0.4997 - val_accuracy: 0.7543
Epoch 3/3
263/263 [==============================] - 100s 381ms/step - loss: 0.4385 - accuracy: 0.8092 - val_loss: 0.5256 - val_accuracy: 0.7457


In [12]:
# Evaluate the model
loss, accuracy = model.evaluate(
    {'input_ids': x_test_tokens['input_ids'], 'attention_mask': x_test_tokens['attention_mask']},
    y_test
)
print(f'Accuracy of BERT model: {accuracy:.2f}')

37/37 [==============================] - 7s 124ms/step - loss: 0.6043 - accuracy: 0.7391
Accuracy of BERT model: 0.74


## Using LSTM

In [21]:
tokenizer = Tokenizer(num_words=5000, lower=True, oov_token='UNK')
tokenizer.fit_on_texts(df['cleaned_text'].values)

In [23]:
max_len = max(df["document_length"])
max_len

69

In [24]:
max_words = len(tokenizer.word_index)
print(max_words)

10373


In [25]:
seq = tokenizer.texts_to_sequences(df["cleaned_text"])

In [26]:
pad_sequences(tokenizer.texts_to_sequences(df["cleaned_text"]),maxlen=max_len)

array([[   0,    0,    0, ..., 3554,  510,  302],
       [   0,    0,    0, ..., 3555,  166, 2812],
       [   0,    0,    0, ...,  230,   42, 2813],
       ...,
       [   0,    0,    0, ...,  342,   66,  924],
       [   0,    0,    0, ..., 2881,   17,    4],
       [   0,    0,    0, ...,  197, 3547,  200]], dtype=int32)

In [39]:
x = pad_sequences(tokenizer.texts_to_sequences(df["cleaned_text"]),maxlen=max_len)
y = df['Sentiment_encoded']

In [40]:
x.shape

(5841, 69)

In [41]:
y = to_categorical(y, num_classes=70)
y.shape

(5841, 70)

In [62]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y)

In [63]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4672, 69), (1169, 69), (4672, 70), (1169, 70))

In [46]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [64]:
model = Sequential()

model.add(Embedding(max_words+1,100,input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5))
model.add(Dense(70, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [65]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [66]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_2                  │ ?                           │     0 (unbuilt) │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [67]:
history = model.fit(x_train, y_train, batch_size=25, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 27s 127ms/step - accuracy: 0.4605 - loss: 2.2708 - val_accuracy: 0.5355 - val_loss: 0.9944
Epoch 2/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step - accuracy: 0.5511 - loss: 0.9797 - val_accuracy: 0.5355 - val_loss: 0.9729
Epoch 3/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.5650 - loss: 0.9434 - val_accuracy: 0.6236 - val_loss: 0.8977
Epoch 4/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 42s 132ms/step - accuracy: 0.6984 - loss: 0.7416 - val_accuracy: 0.6210 - val_loss: 0.8979
Epoch 5/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 41s 133ms/step - accuracy: 0.7489 - loss: 0.6287 - val_accuracy: 0.6553 - val_loss: 0.8780
Epoch 6/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 25s 132ms/step - accuracy: 0.7982 - loss: 0.5201 - val_accuracy: 0.6510 - val_loss: 0.9076
Epoch 7/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 41s 133ms/step - accuracy: 0.8205 - loss: 0.4509 - val_accuracy: 0.6416 - val_loss: 0.9070
Epoch 8/15
187/187 ━━━━━━━━━━━━━━━━━━━━ 40s 131ms/step - accuracy: 0.8517 - loss: 0

In [68]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Accuracy of LSTM model: {accuracy:.2f}')

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6154 - loss: 1.4149
Accuracy of LSTM model: 0.63


In [69]:
new_sentences = ["The company achieved a 20% increase in revenue this quarter compared to the previous quarter, driven by strong demand for our new product line."]
seq = tokenizer.texts_to_sequences(new_sentences)
padded = pad_sequences(seq, maxlen=max_len)
predictions = model.predict(padded)

sentiment_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

for sentence, sentiment in zip(new_sentences, sentiment_labels):
  print(f"Sentence: {sentence}, Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Sentence: The company achieved a 20% increase in revenue this quarter compared to the previous quarter, driven by strong demand for our new product line., Predicted Sentiment: positive


In [71]:
while True:
  new_sentence = input("Enter a new sentence to predict its sentiment: ")
  if new_sentence == "":
    break
  seq = tokenizer.texts_to_sequences([new_sentence])
  padded = pad_sequences(seq, maxlen=max_len)
  predictions = model.predict(padded)
  sentiment_label = label_encoder.inverse_transform(np.argmax(predictions, axis=1))[0]
  print(f"Sentence: {new_sentence}, Predicted Sentiment: {sentiment_label}")

print("Exiting sentiment prediction loop.")

Enter a new sentence to predict its sentiment: Earnings per share fell to -$0.50 this quarter, down from $0.75 in the previous quarter, reflecting poor financial performance and operational challenges.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Sentence: Earnings per share fell to -$0.50 this quarter, down from $0.75 in the previous quarter, reflecting poor financial performance and operational challenges., Predicted Sentiment: negative
Enter a new sentence to predict its sentiment: The recent launch of our flagship product exceeded sales expectations by 30%, contributing significantly to our overall revenue growth
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Sentence: The recent launch of our flagship product exceeded sales expectations by 30%, contributing significantly to our overall revenue growth, Predicted Sentiment: positive
Enter a new sentence to predict its sentiment: The company’s revenue remained consistent at $10 million this quarter, similar to the previous quarter’s performance
1/1